# Question & Answering with ArXiV papers at scale
This notebook is about neural question and answering using transformers models (ALBERT) at SCALE. The below approach is capable to perform Q&A across millions of documents in few seconds.

I will be using ArXiV's papers abstracts to do Q&A at this point it time as I do not have access to actual PDF texts. But - the same approach can be followed to seek answers from actual text in place of just the abstracts. 

I will post another notebook when I get my hands on the actual paper's texts. Now let's dive in...

### Activate GPU support

In [1]:
import torch
torch.cuda.is_available()

True

If successful, the output of the cell above should print `True`. Note that Google Colaboratory also offers [TPU](https://cloud.google.com/tpu/) support. These *Tensor Processing Units* are specifically designed for machine learning tasks and may outperform conventional GPUs. While support for TPUs in PyTorch is still pending, [tensorflow](https://www.tensorflow.org/) models may benefit from using TPUs (see [this tutorial](https://colab.research.google.com/notebooks/tpu.ipynb)).

### Useful commands

Within the notebook environment, you can not only execute Python code, but also bash commands by prepending a `!`. For example, you can install new Python packages via the package manager `pip`. Here, we just check the installed version of PyTorch:

In [2]:
!pip show torch

Name: torch
Version: 1.6.0+cu101
Summary: Tensors and Dynamic neural networks in Python with strong GPU acceleration
Home-page: https://pytorch.org/
Author: PyTorch Team
Author-email: packages@pytorch.org
License: BSD-3
Location: /usr/local/lib/python3.6/dist-packages
Requires: future, numpy
Required-by: torchvision, torchtext, fastai


Another useful command is `!kill -9 -1`. It will reset all running kernels and free up memory (including GPU memory). Furthermore, there are a few commands to have a closer look on the hardware spcifications, i.e. to get information about the installed CPU and GPU:

In [3]:
!lscpu |grep 'Model name'

Model name:          Intel(R) Xeon(R) CPU @ 2.00GHz


In [4]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-902f050f-653d-99f2-f055-1bf112442b87)


In addition, you can check the available RAM and HDD memory:

In [5]:
!cat /proc/meminfo | grep 'MemAvailable'

MemAvailable:   12435516 kB


In [6]:
!df -h / | awk '{print $4}'

Avail
35G


Finally, one can execute the following command to get a live update on the GPU usage. This is useful to check how much of the GPU memory is in use to optimize the batchsize for training. Note that whenever the training routine in a notebook is still running, you need to execute this command in another Colaboratory notebook to get an instant response:

In [7]:
!nvidia-smi

Sat Sep 12 04:40:43 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.66       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   52C    P8    10W /  70W |     10MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

### Mount Google Drive
Another important prerequisite for training our neural network is a place to save checkpoints of the trained model and to store obtained training data. Colaboratory provides convenient access to Google Drive via the `google.colab` Python module. The following command will mount your Google Drive contents to the folder path `/content/gdrive` on the Colaboratory instance. For authentication, you have to click the generated link and paste the authorization code into the input field:

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

### Download the arxiv meta data witg gsutil
We will need gsutil utility from google cloud sdk. Firstly, you need to authenticate yourself in Colab. Once you run the code below, it will ask you to follow a link to login and enter an access token that you receive upon successful login.


In [9]:
from google.colab import auth
auth.authenticate_user()

We would be using the gsutil command to upload and download files. So we first need to install the GCloud SDK.

In [10]:
!curl https://sdk.cloud.google.com | bash
!gcloud init

Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
google-cloud-sdk/lib/surface/iam/service_accounts/sign_jwt.py
google-cloud-sdk/lib/surface/iam/service_accounts/undelete.yaml
google-cloud-sdk/lib/surface/iam/service_accounts/update.yaml
google-cloud-sdk/lib/surface/iam/simulator/__init__.py
google-cloud-sdk/lib/surface/iam/simulator/replay_recent_access.py
google-cloud-sdk/lib/surface/iam/workload_identity_pools/__init__.py
google-cloud-sdk/lib/surface/iam/workload_identity_pools/create.yaml
google-cloud-sdk/lib/surface/iam/workload_identity_pools/delete.yaml
google-cloud-sdk/lib/surface/iam/workload_identity_pools/describe.yaml
google-cloud-sdk/lib/surface/iam/workload_identity_pools/list.yaml
google-cloud-sdk/lib/surface/iam/workload_identity_pools/operations/__init__.py
google-cloud-sdk/lib/surface/iam/workload_identity_pools/operations/describe.yaml
google-cloud-sdk/lib/surface/iam/workload_identity_pools/providers/__init__.py
google-cloud-sdk/lib/surfa

### Download the json metadata from the cloud

In [13]:
!gsutil cp -n gs://arxiv-dataset/metadata-v5/arxiv-metadata-oai.json /content/gdrive/arxiv/arxiv-metadata-oai.json
!ls -l /content/gdrive/arxiv

Copying gs://arxiv-dataset/metadata-v5/arxiv-metadata-oai.json...
- [1 files][  4.2 GiB/  4.2 GiB]   42.4 MiB/s                                   
Operation completed over 1 objects/4.2 GiB.                                      
total 4398424
-rw-r--r-- 1 root root 4503980272 Sep 12 04:53 arxiv-metadata-oai.json


### Add progress bar for processing the full dump

In [16]:
!pip install tqdm


### Reading the entire json metadata
This cell may take a minute to run considering the volume of data

In [ ]:
import os
import tqdm
import json

input_file = "/content/gdrive/arxiv/arxiv-metadata-oai.json"

data  = []
with tqdm.tqdm(total=os.path.getsize(input_file)) as pbar:
     with open(input_file, 'r') as f:
          for line in f:
              pbar.update(len(line))
              data.append(json.loads(line))

 34%|███▎      | 1515217436/4503980272 [00:45<01:26, 34734318.02it/s]

I'm limiting my analysis to just 50,000 documents because of the compute limit.

In [ ]:
data = pd.DataFrame(data[:50000])

### Welcome Haystack!

<img src="https://raw.githubusercontent.com/deepset-ai/haystack/master/docs/img/sketched_concepts_white.png">

The secret sauce behind scaling up is **Haystack**. It lets you scale QA models to large collections of documents! 
You can read more about this amazing library here https://github.com/deepset-ai/haystack

For installation: `! pip install git+https://github.com/deepset-ai/haystack.git`

But just to give a background, there are 3 major components to Haystack.
1. **Document Store:** Database storing the documents for our search. We recommend Elasticsearch, but have also more light-weight options for fast prototyping (SQL or In-Memory).
2. **Retriever:** Fast, simple algorithm that identifies candidate passages from a large collection of documents. Algorithms include TF-IDF or BM25, custom Elasticsearch queries, and embedding-based approaches. The Retriever helps to narrow down the scope for Reader to smaller units of text where a given question could be answered.
3. **Reader:** Powerful neural model that reads through texts in detail to find an answer. Use diverse models like BERT, RoBERTa or XLNet trained via FARM or Transformers on SQuAD like tasks. The Reader takes multiple passages of text as input and returns top-n answers with corresponding confidence scores. You can just load a pretrained model from Hugging Face's model hub or fine-tune it to your own domain data.

And then there is **Finder** which glues together a Reader and a Retriever as a pipeline to provide an easy-to-use question answering interface.

In [ ]:
# installing haystack

! pip install git+https://github.com/deepset-ai/haystack.git

In [ ]:
# importing necessary dependencies

from haystack import Finder
from haystack.indexing.cleaning import clean_wiki_text
from haystack.indexing.utils import convert_files_to_dicts, fetch_archive_from_http
from haystack.reader.farm import FARMReader
from haystack.reader.transformers import TransformersReader
from haystack.utils import print_answers

### Setting up DocumentStore
Haystack finds answers to queries within the documents stored in a `DocumentStore`. The current implementations of `DocumentStore` include `ElasticsearchDocumentStore`, `SQLDocumentStore`, and `InMemoryDocumentStore`.

But they recommend `ElasticsearchDocumentStore` because as it comes preloaded with features like [full-text queries](https://www.elastic.co/guide/en/elasticsearch/reference/current/full-text-queries.html), [BM25 retrieval](https://www.elastic.co/elasticon/conf/2016/sf/improved-text-scoring-with-bm25), and [vector storage for text embeddings](https://www.elastic.co/guide/en/elasticsearch/reference/7.6/dense-vector.html).

So - Let's set up a `ElasticsearchDocumentStore`

In [ ]:
! wget https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-7.6.2-linux-x86_64.tar.gz -q
! tar -xzf elasticsearch-7.6.2-linux-x86_64.tar.gz
! chown -R daemon:daemon elasticsearch-7.6.2
 
import os
from subprocess import Popen, PIPE, STDOUT
es_server = Popen(['elasticsearch-7.6.2/bin/elasticsearch'],
                   stdout=PIPE, stderr=STDOUT,
                   preexec_fn=lambda: os.setuid(1)  # as daemon
                  )
# wait until ES has started
! sleep 30

In [ ]:
from haystack.database.elasticsearch import ElasticsearchDocumentStore
document_store = ElasticsearchDocumentStore(host="localhost", username="", password="", index="document")

Once `ElasticsearchDocumentStore` is setup, we will write our documents/texts to the DocumentStore.
* Writing documents to `ElasticsearchDocumentStore` requires a format - **List of dictionaries**
The default format here is: 
`[{"name": "<some-document-name>, "text": "<the-actual-text>"},
{"name": "<some-document-name>, "text": "<the-actual-text>"}
{"name": "<some-document-name>, "text": "<the-actual-text>"}]`

(Optionally: you can also add more key-value-pairs here, that will be indexed as fields in Elasticsearch and can be accessed later for filtering or shown in the responses of the Finder)

* We will use **title** column to pass as `name` and **abstract** column to pass as the `text`

In [ ]:
# Now, let's write the dicts containing documents to our DB.
document_store.write_documents(data[['title', 'abstract']].rename(columns={'title':'name','abstract':'text'}).to_dict(orient='records'))

### Let's prepare Retriever, Reader,  & Finder
**Retrievers** help narrowing down the scope for the Reader to smaller units of text where a given question could be answered. They use some simple but fast algorithm.

Here: We use Elasticsearch's default BM25 algorithm

In [ ]:
from haystack.retriever.sparse import ElasticsearchRetriever
retriever = ElasticsearchRetriever(document_store=document_store)

A **Reader** scans the texts returned by retrievers in detail and extracts the k best answers. They are based on powerful, but slower deep learning models.

Haystack currently supports Readers based on the frameworks FARM and Transformers. With both you can either load a local model or one from Hugging Face's model hub (https://huggingface.co/models).

Here: a medium sized RoBERTa QA model using a Reader based on FARM (https://huggingface.co/deepset/roberta-base-squad2)

In [ ]:
reader = FARMReader(model_name_or_path="deepset/roberta-base-squad2", use_gpu=True, context_window_size=500)

And finally:  The **Finder** sticks together reader and retriever in a pipeline to answer our actual questions. 

In [ ]:
finder = Finder(reader, retriever)

### And we're done !
Below is the list of questions that I was asking the model and the results were pleasing.

In [ ]:
sample_questions = ["What do we know about Bourin and Uchiyama?",
       "How is structure of event horizon linked with Morse theory?",
       "What do we know about symbiotic stars"]

In [ ]:
prediction = finder.get_answers(question="What do we know about symbiotic stars", top_k_retriever=10, top_k_reader=2)
result = print_answers(prediction, details="minimal")